In [1]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [2]:
#best_model = YOLO("best_model.pt")

#print(f"The task type of the model is: {best_model.task}")

In [2]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [3]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("/home/under1/Detect/jeongui/sagittal/test/runs/detect/train/train_base/weights/best.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/sagittal/test/test.yaml",
    epochs=30,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=48447) Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=48447) engine/trainer: task=detect, mode=train, model=/home/under1/Detect/jeongui/sagittal/test/runs/detect/train/train_base/weights/best.pt, data=/home/under1/Detect/jeongui/sagittal/test/test.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
 18%|█▊        | 1.11M/6.25M [00:00<00:00, 11.6MB/s]
 72%|███████▏  | 4.52M/6.25M [00:00<00:00, 46.9MB/s]
100%|██████████| 6.25M/6.25M [00:00<00:00, 32.2MB/s]
100%|██████████| 6.25M/6.25M [00:00<00:00, 43.2MB/s]
100%|██████████| 6.25M/6.25M [00:00<00:00, 47.7MB/s]
train: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]


(_tune pid=48447) AMP: checks passed ✅
(_tune pid=48446) Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB) [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_tune pid=48449) engine/trainer: task=detect, mode=train, model=/home/under1/Detect/jeongui/sagittal/test/runs/detect/train/train_base/weights/best.pt, data=/home/under1/Detect/jeongui/sagittal/test/test.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, ove

  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 2x across cluster]
100%|██████████| 6.25M/6.25M [00:00<00:00, 30.2MB/s] [repeated 3x across cluster]
(_tune pid=48447) INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


(_tune pid=48447) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=48448) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=48449) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=48446) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=48447) Plotting labels to runs/detect/tune/tune_raytune/labels.jpg... 
(_tune pid=48446) AMP: checks passed ✅ [repeated 3x across cluster]
(_tune pid=48447) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.07851340658107606' and 'momentum=0.95378119686571' and determ

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.84it/s]


(_tune pid=21515) 
(_tune pid=21515) 30 epochs completed in 0.222 hours.
(_tune pid=21515) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.2MB
(_tune pid=21515) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.2MB
(_tune pid=21515) 
(_tune pid=21515) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=21515) Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=21515) Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]
      30/30       2.1G     0.0115      5.185      1.214         16        640:  86%|████████▌ | 12/14 [00:01<00:00,  7.48it/s] [repeated 19x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  3.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]


(_tune pid=21515)               negative         30         30      0.583      0.933      0.742      0.451
(_tune pid=21515)               positive         22         24      0.358      0.833      0.539      0.323
(_tune pid=21515)                    all         52         54      0.471      0.883      0.641      0.387 [repeated 2x across cluster]
(_tune pid=21515) Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 2.8ms postprocess per image
(_tune pid=21515) Results saved to runs/detect/tune/tune_raytune


2024-07-31 16:06:38,102	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/sagittal/test/runs/detect/tune/_tune_2024-07-31_15-32-59' in 0.0151s.
2024-07-31 16:06:38,119	ERROR tune.py:1037 -- Trials did not complete: [_tune_2a884_00000]
2024-07-31 16:06:38,121	INFO tune.py:1041 -- Total run time: 2019.01 seconds (2018.93 seconds for the tuning loop).


In [4]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.03545934464001868, 'lrf': 0.1269818219816718, 'momentum': 0.6400495135113041, 'weight_decay': 0.0004901573182214308, 'warmup_epochs': 2.7704237309978597, 'warmup_momentum': 0.12211803848513114, 'box': 0.15555754918222586, 'cls': 1.2881973698516505, 'hsv_h': 0.06992507014486865, 'hsv_s': 0.5432212691180824, 'hsv_v': 0.05684205101839689, 'degrees': 20.054912707992646, 'translate': 0.22725394216708483, 'scale': 0.3826875883904954, 'shear': 1.7992683436488255, 'perspective': 0.0006875648974584802, 'flipud': 0.03799979682933041, 'fliplr': 0.31194858157426597, 'bgr': 0.12684554613401888, 'mosaic': 0.04637648204752387, 'mixup': 0.9751916199254702, 'copy_paste': 0.6268482131422615, 'data': '/home/under1/Detect/jeongui/sagittal/test/test.yaml', 'epochs': 30, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.55268, 'metrics/recall(B)': 0.89783, 'metrics/mAP50(B)': 0.72175, 'metrics/mAP50-95(B)': 0.36648, 'val/box_loss': 0.03714, 'v

In [5]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.03545934464001868, 'lrf': 0.1269818219816718, 'momentum': 0.6400495135113041, 'weight_decay': 0.0004901573182214308, 'warmup_epochs': 2.7704237309978597, 'warmup_momentum': 0.12211803848513114, 'box': 0.15555754918222586, 'cls': 1.2881973698516505, 'hsv_h': 0.06992507014486865, 'hsv_s': 0.5432212691180824, 'hsv_v': 0.05684205101839689, 'degrees': 20.054912707992646, 'translate': 0.22725394216708483, 'scale': 0.3826875883904954, 'shear': 1.7992683436488255, 'perspective': 0.0006875648974584802, 'flipud': 0.03799979682933041, 'fliplr': 0.31194858157426597, 'bgr': 0.12684554613401888, 'mosaic': 0.04637648204752387, 'mixup': 0.9751916199254702, 'copy_paste': 0.6268482131422615, 'data': '/home/under1/Detect/jeongui/sagittal/test/test.yaml', 'epochs': 30, 'name': 'tune/tune_raytune'}


In [6]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data='test.yaml',
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                val=False,
                                **best_config)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=test.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.03545934464001868' and 'momentum=0.6400495135113041' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0004901573182214308), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      2.13G    0.02349      3.261      1.159          4        640: 100%|██████████| 14/14 [00:01<00:00, 11.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.89it/s]


                   all         52         54      0.587      0.824      0.672      0.312

30 epochs completed in 0.024 hours.
Optimizer stripped from runs/detect/train/train_raytune/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/train_raytune/weights/best.pt, 6.2MB

Validating runs/detect/train/train_raytune/weights/best.pt...
Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.39it/s]


                   all         52         54      0.587      0.823      0.673      0.312
              negative         30         30      0.689      0.813      0.791      0.394
              positive         22         24      0.486      0.833      0.555      0.231
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train/train_raytune


In [8]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name)

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

train_model = YOLO("/home/under1/Detect/jeongui/sagittal/test/runs/detect/train/train_raytune/weights/best.pt")
train_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="train")

Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  4.73it/s]


                   all        212        225      0.894      0.935      0.971      0.463
              negative        102        109      0.866      0.954      0.973      0.472
              positive        110        116      0.922      0.916      0.969      0.453
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train/train_raytune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f19b0417a60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [9]:
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name)

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

train_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="val")

Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.56it/s]


                   all         52         54      0.587      0.824      0.672      0.311
              negative         30         30       0.69      0.815      0.789      0.391
              positive         22         24      0.485      0.833      0.554      0.231
Speed: 1.1ms preprocess, 3.0ms inference, 0.0ms loss, 15.5ms postprocess per image
Results saved to runs/detect/val/val_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f199c70db40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [10]:
experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name)

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

train_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.70 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/jeongui/sagittal/base/labels/test.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.66it/s]


                   all         70         77      0.439      0.807      0.503      0.231
              negative         35         39      0.492      0.692      0.487      0.239
              positive         35         38      0.385      0.921      0.518      0.224
Speed: 0.6ms preprocess, 4.3ms inference, 0.0ms loss, 13.3ms postprocess per image
Results saved to runs/detect/test/test_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f19d428e410>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804